In [ ]:
import os
from datetime import timedelta
from unittest import TestCase

import inflect
import pytest
import subprocess

from ska_dlm import CONFIG, data_item, dlm_ingest, dlm_migration, dlm_request, dlm_storage
from ska_dlm.dlm_db.db_access import DB
from ska_dlm.dlm_storage.main import persist_new_data_items
from ska_dlm.exceptions import InvalidQueryParameters, ValueAlreadyInDB

Set-up services:

In [ ]:
subprocess.run(['make', 'python-pre-test'], check=True)
!pgrep postgrest

In [ ]:
def _clear_database():
    DB.delete(CONFIG.DLM.dlm_table)
    DB.delete(CONFIG.DLM.storage_config_table)
    DB.delete(CONFIG.DLM.storage_table)
    DB.delete(CONFIG.DLM.location_table)

Define storage location and configure r-clone:

In [ ]:
# @pytest.fixture(scope="function", autouse=True)
# def setup_and_teardown(self):
"""Initialze the tests."""
_clear_database()

# we need a location to register the storage
location_id = dlm_storage.init_location("MyOwnStorage", "Server")
uuid = dlm_storage.init_storage(
    storage_name="MyDisk",
    location_id=location_id,
    storage_type="disk",
    storage_interface="posix",
    storage_capacity=100000000,
)
config = '{"name":"MyDisk","type":"local", "parameters":{}}'
dlm_storage.create_storage_config(uuid, config=config)
# configure rclone:
dlm_storage.rclone_config(config)
    # yield
    # _clear_database()

Intialize a new data_item:

In [ ]:
def test_init():
    """Test data_item init."""
    engine = inflect.engine()
    success = True
    for i in range(1, 51, 1):
        ordinal = engine.number_to_words(engine.ordinal(i))
        uid = dlm_ingest.init_data_item(f"this/is/the/{ordinal}/test/item")
        if uid is None:
            success = False
    if success:
        print("Success")

test_init()

Ingest a data item:

In [ ]:
# def test_ingest_data_item(self):
"""Test the ingest_data_item function."""
uid = dlm_ingest.ingest_data_item("/my/ingest/test/item", "/LICENSE", "MyDisk")
if len(uid) == 36:
    print('Pass')
else:
    print('Fail')
# assert len(uid) == 36

Register a data item:

In [ ]:
uid = dlm_ingest.register_data_item("/my/ingest/test/item2", "/LICENSE", "MyDisk")
assert len(uid) == 36
with pytest.raises(ValueAlreadyInDB, match="Item is already registered"):
    dlm_ingest.register_data_item("/my/ingest/test/item2", "/LICENSE", "MyDisk")

Query for all expired data_items:

In [ ]:
# empty set:
result = dlm_request.query_expired()
success = len(result) == 0
assert success

In [ ]:
# returns results:
test_init()
offset = timedelta(days=1)
result = dlm_request.query_expired(offset)
success = len(result) != 0
assert success

In [ ]:
# def test_location_init(self):
"""Test initialisation on a location."""
# This returns an empty string if unsuccessful
# with pytest.raises(InvalidQueryParameters):
#     dlm_storage.init_location()
dlm_storage.init_location("TestLocation", "SKAO Data Centre")
location = dlm_storage.query_location(location_name="TestLocation")[0]
assert location["location_type"] == "SKAO Data Centre"

Update a data_item record with a file location, a state and a phase:

In [ ]:
fname = "dlm_test_file_1.txt"
with open(fname, "w", encoding="UTF-8") as tfile:
    tfile.write("Welcome to the great DLM world!")
fpath = os.path.abspath("dlm_test_file.txt")
fpath = fpath.replace(f"{os.environ['HOME']}/", "")
uid = dlm_ingest.init_data_item(item_name="this/is/the/first/test/item")
storage_id = dlm_storage.query_storage(storage_name="MyDisk")[0]["storage_id"]
data_item.set_uri(uid, fpath, storage_id)
# check that the uri of the data item (with uid = uid) is equal to fpath
assert dlm_request.query_data_item(uid=uid)[0]["uri"] == fpath
data_item.set_state(uid, "READY")
data_item.set_phase(uid, "PLASMA")
# query all data items with uid=uid, and check their state & phase is what we set it to be:
items = dlm_request.query_data_item(uid=uid)
assert len(items) == 1
assert items[0]["item_state"] == "READY"
assert items[0]["item_phase"] == "PLASMA"
print("Success")
os.unlink(fname)

Delete the payload of a data_item:

In [ ]:
fpath = "dlm_test_file_2.txt"
with open(fpath, "w", encoding="UTF-8") as tfile:
    tfile.write("Welcome to the great DLM world!")
# add the file to the database:
storage_id = dlm_storage.query_storage(storage_name="MyDisk")[0]["storage_id"]
uid = dlm_ingest.ingest_data_item(fpath)
queried_uid = dlm_request.query_data_item(item_name=fpath)[0]["uid"]
assert uid == queried_uid
dlm_storage.delete_data_item_payload(uid)
data_item.set_uri(uid, fpath, storage_id)
data_item.set_state(uid, "DELETED")
assert dlm_request.query_data_item(item_name=fpath)[0]["uri"] == fpath
assert dlm_request.query_data_item(item_name=fpath)[0]["item_state"] == "DELETED"

Tear down services:

In [ ]:
_clear_database()

In [ ]:
!make python-post-test